In [3]:
from flask import Flask, render_template, request, send_file, jsonify
import cv2
import numpy as np
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
PROCESSED_FOLDER = 'processed'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['image']
    filename = secure_filename(file.filename)
    filepath = os.path.join(UPLOAD_FOLDER, filename)
    file.save(filepath)
    return jsonify({'filename': filename})

@app.route('/process/<operation>/<filename>')
def process(operation, filename):
    filepath = os.path.join(UPLOAD_FOLDER, filename)
    image = cv2.imread(filepath)

    if operation == 'grayscale':
        processed = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif operation == 'canny':
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        processed = cv2.Canny(gray, 50, 150)
    elif operation == 'sobel':
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        processed = cv2.convertScaleAbs(sobelx) + cv2.convertScaleAbs(sobely)
    elif operation == 'gaussian':
        processed = cv2.GaussianBlur(image, (5, 5), 0)
    elif operation == 'median':
        processed = cv2.medianBlur(image, 5)
    elif operation == 'contours':
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        processed = image.copy()
        cv2.drawContours(processed, contours, -1, (0, 255, 0), 2)
    else:
        return jsonify({'error': 'Unknown operation'})

    out_path = os.path.join(PROCESSED_FOLDER, f'{operation}_{filename}')
    if len(processed.shape) == 2:
        cv2.imwrite(out_path, processed)
    else:
        cv2.imwrite(out_path, cv2.cvtColor(processed, cv2.COLOR_RGB2BGR))

    return send_file(out_path, mimetype='image/jpeg')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1